# Embedtree2
Notebook to compile the so far research of nvgtt.

In [227]:
import requests
import nltk
from bs4 import BeautifulSoup
import re
import pickle
import urllib

### Dict Storage class to be used to load and save dicts
It saves data in pickle format and works with lower case only to save memory

In [228]:
class DictStorage(dict):
    
    def __init__(self, filename):
        self.filename = filename + ".pickle"
        
        try:
            with open(self.filename, mode='r') as pickle_file:
                dict_list = pickle.load(pickle_file)
        except IOError:
            dict_list = []
            print("Failed to open " + self.filename + ". Created empty dict.")
        finally:
            dict.__init__(self, dict_list)
    
    def save(self):
        with open(self.filename, mode='w') as pickle_file:
            pickle.dump(self.items(), pickle_file)
            
  #  def __setitem__(self, key, value):
 #       #Ensure data is string 
 #       super(DictStorage, self).__setitem__(key.lower(), value)

 #   def __getitem__(self, key):
#        #Ensure data is string 
 #       return super(DictStorage, self).__getitem__(key.lower())
    
#    def has_key(self, key):
#        #Ensure data is string 
#        return super(DictStorage, self).has_key(key.lower())

In [229]:
def test_dictstorage():
    test1 = DictStorage("ae2")
    print(test1.items())
    print(test1.save())
    test1['LUCAS'] = 128
    print(test1.items())
    print(test1.save())
    print(test1['lucas'])
    print(test1['LUCAS'])
    
#test_dictstorage()

### Function to get wikipedia data

In [230]:
def get_page(page, lang="en"):
    """Function to retrieve a wikipedia page in html form, with its sections"""

    # https://en.wikipedia.org/w/api.php?action=parse&redirects&page=fluid_mechanics
    """
    wikipediaApiUrl = "https://" + lang + ".wikipedia.org/w/api.php"
    
    pageParams = {
        'action': 'parse', 
        'redirects': True,
        'page': urllib.unquote(page),
        'format': 'json',
        'prop':'text|displaytitle'
    }

    pageData = requests.get(wikipediaApiUrl, pageParams).json()
    """    
    pageParams = [
        'action=parse',
        'redirects',
        'format=json',
        'prop=text|displaytitle',
        'page=' + page
    ]
    
    wikipediaApiUrl = "https://" + lang + ".wikipedia.org/w/api.php?" + "&".join(pageParams)

    pageData = requests.get(wikipediaApiUrl).json()
    
    if not 'parse' in pageData:
        print page 
        print urllib.unquote(page)
        print(pageData)
        raise "Error while getting page " + page


    docHtml = BeautifulSoup(pageData['parse']['text']['*'], 'html.parser')

    #Split document by its sections
    docSections = __splitIntoSections__(docHtml)

    structPageData = {
        'title': pageData['parse']['title'],
        'pageid': pageData['parse']['pageid'],
        'full': docHtml,
        'sections': docSections
    }

    return structPageData


def __splitIntoSections__(htmlObj):
    """Function to split html document in sections (use h2 tags as divisors)"""

    #Init var to store sections
    sectionObjs = [[]]

    for tag in htmlObj.children:
        #Start new section in case the tag is h2
        if tag.name == 'h2':
            sectionObjs.append([])

        #If it is a valid tag (invalid tags has no 'name' property)
        if tag.name != None:
            sectionObjs[len(sectionObjs) - 1].append(tag)

    return sectionObjs
    

In [231]:
#print get_page("C%2B%2B")['title']

In [232]:
def get_page_and_parse(page):
    """Function to treat the data, remove unecessary things etc."""
    
    page_data = get_page(page)
    
    soup = page_data['full']
        
    #Clear table of contents if any
    for node in soup.findAll(id='toc'):
        node.decompose()

    #Clear top info table if any
    for node in soup.findAll(class_='ambox'):
        node.decompose()

    #Clear info box if any
    for node in soup.findAll(class_='infobox'):
        node.decompose()

    #Clear verticalbox if any
    for node in soup.findAll(class_='vertical-navbox'):
        node.decompose()
        
    #Clear navbox if any
    for node in soup.findAll(class_='navbox'):
        node.decompose()
        
    return page_data

In [233]:
blocked_link_terms = set([
    "(disambiguation)", #Not interested in disambiguation pages
    ":" #Pages with colon are offen special pages. Not sure if there is articles with colon
])

def get_page_links(page_data):
    """Function to get links """
    #page_data = get_page_and_parse(page)
    
    links = list()
    
    for link in page_data['full'].findAll("a"):
        
        #If the a tag has no href attr, skip it
        if not link.has_attr("href"):
            continue
            
        #If the href does not starts with "/wiki/", skip it
        if link['href'].find("/wiki/") != 0:
            continue
            
        #Check if some blocked term is present in the href, if so, skip the link
        skip_link = False
        for term in blocked_link_terms:
            if link['href'].find(term) != -1:
                skip_link = True
                break
        if skip_link:
            continue
        
        #Get only the link portion
        #We MUST NOT use last index of / to get the path cause some titles like TCP/IP, have bar in the title
        #We should use the '/wiki/' string length
        linkHref = link['href'][6:]
        
        #Remove hashtag from url if any
        hashIndex = linkHref.find("#")
        if hashIndex != -1:
            linkHref = linkHref[:hashIndex]
            
        linkText = link.get_text()
            
        links.append((linkHref, linkText))
            
    return links#, n_valid_links

## Main()

In [234]:
wikisyn = DictStorage("wikisyn") #Storage for link synoms
href_to_pageid = DictStorage("href_to_pageid") #Storage lookup table of hrefs and pageids
pageid_to_href = DictStorage("pageid_to_href") #Storage lookup table of pageid and hrefs
pageid_to_page_links = DictStorage("pageid_to_page_links") #Storage for page links
pageid_to_page_text = DictStorage("pageid_to_page_text") #Storage for page texts

Failed to open wikisyn.pickle. Created empty dict.
Failed to open href_to_pageid.pickle. Created empty dict.
Failed to open pageid_to_href.pickle. Created empty dict.
Failed to open pageid_to_page_links.pickle. Created empty dict.
Failed to open pageid_to_page_text.pickle. Created empty dict.


In [235]:
#Function to save page data to storages
def save_page_data(page_href, page_data):
    page_title = page_data['title']
    page_id = page_data['pageid']
    page_text = page_data['full'].get_text()
    
    #Save page id lookup tables
    href_to_pageid[page_href] = page_id
    href_to_pageid[page_title] = page_id
    href_to_pageid.save()
    
    if not pageid_to_href.has_key(page_id):
        pageid_to_href[page_id] = set()
    pageid_to_href[page_id].add(page_href)
    pageid_to_href[page_id].add(page_title)
    pageid_to_href.save()
    
    #Save page text
    pageid_to_page_text[page_id] = page_text
    pageid_to_page_text.save()
    
    #save page links and wikisyn
    page_links = get_page_links(page_data)
    pageid_to_page_links[page_id] = set()
    for link_href, link_text in page_links:
        pageid_to_page_links[page_id].add(link_href)
        if not wikisyn.has_key(link_href):
            wikisyn[link_href] = set()
        wikisyn[link_href].add(link_text)
    pageid_to_page_links.save()
    wikisyn.save()    

In [236]:
def get_pageid(page):
    
    #Check if the page is in the redirects table, if not, download it and register it
    if not href_to_pageid.has_key(page):
        print("Page not found. Downloading and registering it...")
        page_data = get_page_and_parse(page)
        save_page_data(page, page_data)
        print("Done.")
        
    return href_to_pageid[page]

In [237]:
target_page = urllib.quote("JavaScript")
target_id = get_pageid(target_page)
target_links = pageid_to_page_links[target_id]
#target_links_ids = set()
MUST FIND HOW TO DEAL WITH  NETWORKX NODES NAMES
WE WILL NEED TO ALWAYS DOWNLOAD ONE DEEP MORE TO GET LINKS ID INFOS

MAYBE KEEP TRACK OF ARTICLES TITLES
#for i, link in enumerate(target_links):
    #print("Working on " + link + ". " + str((i+1)) + "/" + str(len(target_links)))
    #target_links_ids.add(get_pageid(link))


Page not found. Downloading and registering it...
Done.


In [244]:
def get_links_score(page):
    """Function to cross a list of links with a text, setting scores."""
    
    pageid = href_to_pageid[page]
    page_links = pageid_to_page_links[pageid]
    page_text = pageid_to_page_text[pageid]
    
    links_score = dict()
    
    norm_fact = 0
    
    for link_href in page_links:
        links_score[link_href] = 0
        for l_text in wikisyn[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            links_score[link_href] += len(matches) 
            norm_fact += len(matches)
            
    norm_links_score = dict(map(lambda a: [a[0], float(a[1])/norm_fact], links_score.items()))
            
    return norm_links_score

v_sum = 0
links_score = get_links_score("JavaScript")
for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    print(k,wikisyn[k],v)
    v_sum += v
print v_sum

(u'Subroutine', set([u'function']), 0.04969418960244648)
(u'Object_(computer_science)', set([u'object']), 0.03669724770642202)
(u'ECMAScript', set([u'ECMAScript \xa7\xa0Version correspondence', u'ECMAScript \xa7\xa0Versions', u'ECMAScript']), 0.034403669724770644)
(u'Java_(programming_language)', set([u'Java (programming language)', u'Java']), 0.02522935779816514)
(u'Microsoft', set([u'Microsoft']), 0.02522935779816514)
(u'Microsoft_Outlook', set([u'Microsoft']), 0.02522935779816514)
(u'Mozilla', set([u'Mozilla']), 0.02217125382262997)
(u'Prototype_JavaScript_Framework', set([u'Prototype']), 0.019877675840978593)
(u'Computer_program', set([u'script']), 0.0191131498470948)
(u'International_Standard_Book_Number', set([u'ISBN']), 0.016819571865443424)
(u'Web_browser', set([u'Web browsers', u'Web browser']), 0.01529051987767584)
(u'Google', set([u'Google']), 0.01529051987767584)
(u'Programming_language', set([u'programming language']), 0.01452599388379205)
(u'Internet_Explorer', set([u'Int

In [ ]:
page_links = get_page_links(page_data)

for k, v in page_links:
    print(k,v)

In [240]:
links_score = get_page_links_score(page_links, page_data['full'].get_text())

NameError: name 'page_links' is not defined

In [10]:
total_links = 0
for k, v in links_score.items():
    total_links += v

for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    print(k,page_links[k],float(v),float(v)/total_links)

(u'django_(disambiguation)', set([u'django']), 136.0, 0.28936170212765955)
(u'python_(programming_language)', set([u'python']), 22.0, 0.04680851063829787)
(u'international_standard_book_number', set([u'isbn']), 8.0, 0.01702127659574468)
(u'ruby_(programming_language)', set([u'ruby']), 4.0, 0.00851063829787234)
(u'model-view-controller', set([u'mvc']), 4.0, 0.00851063829787234)
(u'jython', set([u'jython']), 4.0, 0.00851063829787234)
(u'd_(programming_language)', set([u'd']), 4.0, 0.00851063829787234)
(u'asp.net_mvc', set([u'mvc']), 4.0, 0.00851063829787234)
(u'web_framework', set([u'web frameworks', u'web framework']), 3.0, 0.006382978723404255)
(u'asp.net_core', set([u'core']), 3.0, 0.006382978723404255)
(u'http', set([u'http']), 3.0, 0.006382978723404255)
(u'javascript', set([u'javascript']), 3.0, 0.006382978723404255)
(u'google_app_engine', set([u'google app engine']), 3.0, 0.006382978723404255)
(u'firebird_(database_server)', set([u'firebird']), 3.0, 0.006382978723404255)
(u'django_

In [138]:
#page_data = get_links_score()
#for d in page_data.iteritems():
    #print(d)

In [238]:
#print(page_data['full'].get_text())